# Step-by-step walkthrough of Etch contract development and deployment

### **Installation (for first-time use)**

Install Anaconda (if not done already): [Download Link](<https://docs.anaconda.com/anaconda/install/>)

Setup Python env "etch" for Etch contract development. It is currently tested for Python3.6.

```
conda update conda -y
conda create -n etchenv python=3.6 -y
conda activate etchenv
```

Pip install Ledger Python API. The latest version is currently alpha v0.10.x. [Resource](https://docs.fetch.ai/getting-started/python-api-install/)

`pip install -U fetchai-ledger-api`

Alternatively you can install directly from source:

```
git clone https://github.com/fetchai/ledger-api-py.git
cd ledger-api-py
pip uninstall fetchai-ledger-api
```

Add etchenv as an iPython kernel and restart Jupyter Notebook. That way you can see "etchenv" as a kernel option.

`python -m ipykernel install --user --name etchenv --display-name "etch"`

There are two ways settting up your Etch Ledger locally:
    1) From Source
    2) From Docker

***Setting up Ledger from Source***

Install the Fetch ledger at your workspace directory. See see Fetch [docs](https://docs.fetch.ai/) "Getting Started" section for instructions to do so. Currently, it is compatible with `MacOS`, `Ubuntu`, and `Redhat` systems.

Once built (just `make -j 4 constellation`), move to the following directory:

`cd ledger/build/apps/constellation`

Run the Fetch ledger on your local testnode with the following command:

`./constellation -port 8100 -block-interval 3000 -standalone`

The output will show the following in console:

```
F E ╱     Constellation v0.4.1-rc1
   T C     Copyright 2018-2019 (c) Fetch AI Ltd.
     H

[ 2019-06-03 16:55:20.215, # 1 INFO  :                                main ] Configuration:

port......................: 8100
network mode..............: Standalone
num executors.............: 1
num lanes.................: 1
num slices................: 500
bootstrap.................: 0
discoverable..............: 0
host name.................:
external address..........: 127.0.0.1
db-prefix.................: node_storage
interface.................: 127.0.0.1
mining....................: Yes
tx processor threads......: 12
shard verification threads: 12
block interval............: 3000ms
max peers.................: 3
peers update cycle........: 0ms
peers.....................:
manifest.......:
 - HTTP/0: tcp://127.0.0.1:8100 (8100)
 - CORE/0: tcp://127.0.0.1:8101 (8101)
 - Lane/0: tcp://127.0.0.1:8110 (8110)
```

***Setting up Ledger from Docker***

Install Docker CE engine locally (if you haven't already). [Link](https://docs.docker.com/install/linux/docker-ce/ubuntu/)

Pull latest Constellation image from Docker Hub. [Link](https://hub.docker.com/r/fetchai/constellation)

`docker pull fetchai/constellation`

Run the Fetch Ledger via Docker. [Link](https://github.com/fetchai/docker-constellation/blob/master/docs/description.md)

`docker run -p 8100:8100 fetchai/constellation -port 8100 -block-interval 3000 -standalone`

The output should appear in console:

```
 F E ╱     Constellation v0.9.1
   T C     Copyright 2018-2019 (c) Fetch AI Ltd.
     H     

[I] 2019/11/27 16:19:14 | main                           : Input Configuration:
lanes................: 1
slices...............: 500
block-interval.......: 3000
standalone...........: Yes
private-network......: No
db-prefix............: node_storage
port.................: 8100
peers................: 
external.............: 127.0.0.1
config...............: 
max-peers............: 3
transient-peers......: 1
peers-update-cycle-ms: 0
disable-signing......: No
kademlia-routing.....: Yes
bootstrap............: No
discoverable.........: No
host-name............: 
network..............: 
token................: 
processor-threads....: 4
verifier-threads.....: 4
executors............: 1
load-genesis-file....: No
genesis-file-location: 
experimental.........: 
pos..................: No
max-cabinet-size.....: 10
stake-delay-period...: 5
aeon-period..........: 100
```

The following walkthrough should work regardless of whether the Ledger is built from source or from Docker.

### **Etch Development Walkthrough**

#### **Step 1) Setup Python API and initialize contract owner entity**

Print the Ledger Python API version. Make sure that it is the latest. 

In [1]:
from fetchai.ledger import __version__ as version_string
print(version_string)

0.10.0-a6


Import the following to access Ledger Python API

In [2]:
from fetchai.ledger.api import LedgerApi
from fetchai.ledger.contract import Contract
from fetchai.ledger.crypto import Entity, Address

Set up Python API, using your local config: 

In [3]:
# Constellation config
HOST = '127.0.0.1'
PORT = 8100

Create an API instance

In [4]:
api = LedgerApi(HOST, PORT)

Create an Entity for owner of the Etch contract, where contains owner's public/private keypair as well as its wallet address. In the pet-shop example, the owner can be the shelter iteself.

In [5]:
# Create keypair for the contract owner
owner = Entity()
owner_addr = Address(owner)

In [6]:
print(owner_addr)

2UGxmWcbBW3eEpnmLj2LWJanagwUNBH9aKsguGVu6N7ibYNgJx


(Optional) Save the contract owner's private key to disk with a password

In [7]:
with open('owner_private.key', 'w') as private_key_file:
    owner.dump(private_key_file, password="pass")

#### **Step 2a) Deploy the hello-world contract**

Read-in script from hello.etch

In [ ]:
source = "hello-world/hello.etch"

with open(source, 'r') as file:
    contract_text = file.read()

In [ ]:
print(contract_text)

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [ ]:
contract = Contract(contract_text, owner)

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [ ]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [ ]:
print(api.tokens.balance(address=owner_addr))

Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [ ]:
api.sync(api.contracts.create(owner, contract, 10000))

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### **Step 2b) Interact with the hello-world contract**

Once the contract has been successfully deployed, we can first test it by querying `persistentGreeting`.

In [ ]:
# Printing message
print(contract.query(api, 'persistentGreeting'))

Now, let's change the greeting. First, set the transfer fee amount:

In [ ]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

Then change the greeting recipient:

In [ ]:
# Execute smart contract5
recipient = "Earth"
result = api.sync(contract.action(api, 'changeGreeting', fet_tx_fee, [owner], recipient))

Print the greeting again. Now you see that it has been changed.

In [ ]:
# Printing message
print(contract.query(api, 'persistentGreeting'))

#### **Step 3a) Deploy the pet-shop contract**

Read-in script from adoption.etch

In [92]:
source = "pet-shop/adoption.etch"

with open(source, 'r') as file:
    contract_text = file.read()

Print adoption contract text:

In [93]:
print(contract_text)

//------------------------------------------------------------------------------
//
//   Copyright 2019 Fetch.AI Limited
//
//   Licensed under the Apache License, Version 2.0 (the "License");
//   you may not use this file except in compliance with the License.
//   You may obtain a copy of the License at
//
//       http://www.apache.org/licenses/LICENSE-2.0
//
//   Unless required by applicable law or agreed to in writing, software
//   distributed under the License is distributed on an "AS IS" BASIS,
//   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
//   See the License for the specific language governing permissions and
//   limitations under the License.
//
//------------------------------------------------------------------------------

// This contract require release v0.10.1 alpha or above
// Note that this is experimental functionality

persistent adopters : StructuredData;


@init
function constructor(owner : Address)

    use adopters;
    adopte

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [94]:
contract = Contract(contract_text, owner)

#get contract [digest, address]
contract.name.split('.')

['6c4fae11caa9579d4b877ad0bdc6b9ff9e447d33314b8503fdecaac1135f3abc',
 'uJghCGLMi3am4qQfy64ha9Jb8DW854rzZongNqWmw2KUG7aDf']

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [28]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [95]:
print(api.tokens.balance(address=owner_addr))

39720


Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [96]:
api.sync(api.contracts.create(owner, contract, 10000))

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### **Step 3b) Interact with the pet-shop contract**

In this scenario, let's say there are 4 adopters looking to adopt a pet in the pet-shop. For each adopter, create a new Entity with public/private that enables them to interact with the adoption.etch contract.

In [73]:
adopter1 = Entity()
addr1 = Address(adopter1)
adopter2 = Entity()
addr2 = Address(adopter2)
adopter3 = Entity()
addr3 = Address(adopter3)
adopter4 = Entity()
addr4 = Address(adopter4)

In [98]:
print("Adopter1's address:   " + str(addr1))
print("Adopter2's address:   " + str(addr2))
print("Adopter3's address:   " + str(addr3))
print("Adopter4's address:   " + str(addr4))

Adopter1's address:   2eUsSYuiHg8MqGtsBwosBbpzYf29vsnR7DnPGi4QLLgayFJpts
Adopter2's address:   2KRDuE2mUpzFEt88S7A1pa7LU6NmZCJNGsEyM4vwvc9iXobK3E
Adopter3's address:   2qjpdxEYB2E7fNvSpzWaZ9HFHm1kiUVvJ8h1NvbTunLDQBJwKN
Adopter4's address:   P2B7xhaStFMRpw2LCmRYAG9Wkm4LtwESyRj5JE8uRzd59wei8


Top-up all the adopter entities so they have the FET tokens to pay for tx fees when interacting the the contract.

In [78]:
api.sync(api.tokens.wealth(adopter1, 10000))
api.sync(api.tokens.wealth(adopter2, 10000))
api.sync(api.tokens.wealth(adopter3, 10000))
api.sync(api.tokens.wealth(adopter4, 10000))

Print adopter1's balance to confirm.

In [99]:
print("Adopter1's FET balance:   " + str(api.tokens.balance(address=addr1)))
print("Adopter2's FET balance:   " + str(api.tokens.balance(address=addr2)))
print("Adopter3's FET balance:   " + str(api.tokens.balance(address=addr3)))
print("Adopter4's FET balance:   " + str(api.tokens.balance(address=addr3)))

Adopter1's FET balance:   10000
Adopter2's FET balance:   10000
Adopter3's FET balance:   10000
Adopter4's FET balance:   10000


First, query `getAdopters` function to see which pets are adopted.

In [100]:
# Printing message
result = contract.query(api, 'getAdopters')

No pets have been adopted yet.

In [101]:
print(result)

{}


Next, **adopter1** picks a pet to adopt. Let's say the pet on index 5.

In [102]:
# First, initialize tx fee amount
fet_tx_fee = 500

In [103]:
# Execute smart contract5
pet_id = 5
api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], addr1, pet_id))

Now `getAdopters` function should have one element.

In [104]:
result = contract.query(api, 'getAdopters')
print(result)

{'5': '2eUsSYuiHg8MqGtsBwosBbpzYf29vsnR7DnPGi4QLLgayFJpts'}


Next, **adopter2** and **adopter3** adopt pets on index 7 and 9 respectively. They may need to increase the `fet_tx_fee` in order for the tx to o through.

In [105]:
pet_id = 7
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], addr2, pet_id))
pet_id = 9
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], addr3, pet_id))

In [106]:
result = contract.query(api, 'getAdopters')
print(result)

{'7': '2KRDuE2mUpzFEt88S7A1pa7LU6NmZCJNGsEyM4vwvc9iXobK3E', '9': '2qjpdxEYB2E7fNvSpzWaZ9HFHm1kiUVvJ8h1NvbTunLDQBJwKN', '5': '2eUsSYuiHg8MqGtsBwosBbpzYf29vsnR7DnPGi4QLLgayFJpts'}


Finally, its **adopter4's** turn to adopt a pet, but is a bit clueless about how the system works. First, he tries to adopt a pet on index 16. However, there is no index 16 (only 0-15 is valid) so the assert error msg `assert(petId >= 0 && petId <= 15)` will instead be printed out. So there will be a runtime error in the Python SDK.

In [109]:
pet_id = 16
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], addr4, pet_id))

RuntimeError: Some transactions have failed: 9b3ed967616e03b6c0642e512fc30846e7b4320370d0c22385ab1efb763e3de4:Contract Execution Failure

See the ledger console for the actual `assert` error message:

```
[I] 2019/11/29 18:23:41 | SmartContract                  : Runtime error: runtime error: line 40: Assertion error: Valid indexes are between 0-15.
[W] 2019/11/29 18:23:41 | Executor                       : Transaction execution failed!
[I] 2019/11/29 18:23:41 | BlockCoordinator               : Skipped 3 duplicate messages..
[I] 2019/11/29 18:23:41 | BlockCoordinator               : Merkle Hash3: 
[W] 2019/11/29 18:23:41 | ExecutionManager               : Error executing tx: 0xe9c92243c469898db34feb6c51fb1793fb9b707496ed566549e272f448121c67 status: Contract Execution Failure
[W] 2019/11/29 18:23:41 | ExecutionManager               : Slice 0 Execution Status - Complete: 0 Stalls: 0 Errors: 1 Fatal Errors: 0
```


Next, **adopter4's** tries to adopt a pet on index 9, but is already taken by **adopter3**, as outputed by assert error msg `assert(petId >= 0 && petId <= 15)`.

In [ ]:
pet_id = 9
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], addr4, pet_id))

Now **adopter4** has gotten the memo and decided to settle on adopting the pet on index 11, which is still available.

In [110]:
pet_id = 11
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [owner], addr4, pet_id))

Here is the final `StructuredData` printout with the 4 adopters' addresses and their chosen pet indexes.

In [111]:
result = contract.query(api, 'getAdopters')
print(result)

{'7': '2KRDuE2mUpzFEt88S7A1pa7LU6NmZCJNGsEyM4vwvc9iXobK3E', '11': 'P2B7xhaStFMRpw2LCmRYAG9Wkm4LtwESyRj5JE8uRzd59wei8', '5': '2eUsSYuiHg8MqGtsBwosBbpzYf29vsnR7DnPGi4QLLgayFJpts', '9': '2qjpdxEYB2E7fNvSpzWaZ9HFHm1kiUVvJ8h1NvbTunLDQBJwKN'}


#### **Step 4a) Deploy the simple-open-auction contract**

Now let's try to deploy a more challenging auction contract.

In [ ]:
source = "simple-open-auction/auction.etch"

with open(source, 'r') as file:
    contract_text = file.read()

Print auction contract text:

In [ ]:
print(contract_text)

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [ ]:
contract = Contract(contract_text, owner)

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [ ]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [ ]:
print(api.tokens.balance(address=owner_addr))

Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [ ]:
api.sync(api.contracts.create(owner, contract, 10000))

Check the current block number:

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### **Step 4b) Interact with the simple-open-auction contract**

Create keypair for new entity that interacts with contract. In this case, this will be adopter looking to adopt a pet from the shelter.

In [ ]:
bidder1 = Entity()
bidder_addr1 = Address(adopter1)

print(bidder1)

Top-up the adopter entity as well as the user also needs FET tokens to pyay for tx fees when interacting the the contract.

In [ ]:
api.sync(api.tokens.wealth(bidder1, 10000))

First, query *getAdopters* function to see which pets are adopted.

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

Next, pick a pet to adopt. Let's say pet on index 10.

In [ ]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

make a bid amount:

In [ ]:
bid_amount = 2000 
api.sync(contract.action(api, 'bid', fet_tx_fee, bid_amount)